In [1]:
import os 
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pandasql import sqldf
import numpy as np
import nltk
import re
from wordcloud import WordCloud

In [2]:
nodes = pd.read_csv('/home/manoj/tuhh/data_science_3rd_sem/deep_learning_for_social_analytics/project/deep-learning-for-social-analytics-project/artist_and_music_data/nodes.csv')


In [3]:
nodes.dtypes


spotify_id     object
name           object
followers     float64
popularity      int64
genres         object
chart_hits     object
dtype: object

In [7]:
nodes.isnull().sum()

spotify_id         0
name               4
followers          4
popularity         0
genres             0
chart_hits    136781
dtype: int64

In [8]:
nodes.dropna(subset=['name'], inplace=True)
#delete null names as they are not required.


In [9]:
nodes.dropna(subset=['followers'], inplace=True)
#delete null followers

In [10]:

from concurrent.futures import ProcessPoolExecutor
from langdetect import detect, DetectorFactory

# Ensure consistent results from langdetect
DetectorFactory.seed = 0

def is_english(text):
    try:
        # Fast ASCII check to quickly determine if a text is likely English
        if all(ord(char) < 128 for char in text):
            return True
        return detect(text) == 'en'
    except:
        return False

def filter_english_on_name(df):
    with ProcessPoolExecutor() as executor:
        # Map the language detection function to the 'name' column
        name_english = list(executor.map(is_english, df['name']))
    
    # Return the filtered DataFrame where 'name' is detected as English
    return df[name_english]

nodes_english = filter_english_on_name(nodes)

# Print or further process the filtered DataFrame
print(nodes_english)

                    spotify_id                name  followers  popularity  \
0       48WvrUGoijadXXCsGocwM4           Byklubben     1738.0          24   
1       4lDiJcOJ2GLCK6p9q5BgfK            Kontra K  1999676.0          72   
2       652XIvIBNGg3C0KIGEJWit               Maxim    34596.0          36   
3       3dXC1YPbnQPsfHPVkm1ipj  Christopher Martin   249233.0          52   
4       74terC9ol9zMo8rfzhSOiG       Jakob Hellman    21193.0          39   
...                        ...                 ...        ...         ...   
156417  2ces6d2YsQP1RpGMYpdFy8        David Urwitz     5470.0          29   
156418  6AeznZajNbXUulT7W4tK5l             Darmiko     2022.0          23   
156419  3GEijIjrgb4lPe9WtURBzz            Katriell      268.0           0   
156420  0ldQL0icSoMz9OOZcWG8Zt          Yung Fresh      181.0          19   
156421  1QZqarAGs0Lxx495oNcBnZ       Rakshitha Rao       23.0          24   

                                                   genres  \
0             

In [11]:
nodes_english.shape

(143848, 6)

In [12]:
nodes_english = nodes_english.drop(columns=['spotify_id','chart_hits'])

In [13]:
nodes_english.shape

(143848, 4)

In [14]:

# This is to know the unique genres.
import pandas as pd
import ast


# Convert genre strings to lists
nodes_english['genre_list'] = nodes_english['genres'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else [])

# Collect all unique genres
unique_genres = set(genre for sublist in nodes_english['genre_list'] for genre in sublist)

# Print the unique genres
print("Unique genres:")
for genre in unique_genres:
    print(genre)

Unique genres:
folk metal
heartland rock
classical soprano
modern chamber music
deep liquid bass
jazz trombone
deep freestyle
fourth world
latin jazz
musica de intervencao
black americana
post-post-hardcore
italian hip hop
guam indie
palestinian pop
musica piauiense
bagpipe marching band
pipa
oratory
classical countertenor
water
polish contemporary classical
classic korean pop
belarusian indie
arkansas country
lldm
spanish experimental
corsican folk
dark trap
sovietwave
salsa colombiana
nordnorsk rap
german opera
deep indie rock
schweizer rap
tallava
indie singer-songwriter
rap nacional antigo
classic pakistani pop
musica rondoniense
neue deutsche harte
egyptian alternative
classic tollywood
morna
deep darkpsy
dark wave
kora
sefardi
art song
electroacoustic improvisation
rock caipira
rap congolais
experimental r&b
musica mapuche
musica guineense
laboratorio
kazakh pop
industrial hip hop
rogaland musikk
nz gangsta rap
dutch indie rock
nova musica paulista
greek trap
grupera
korean under

In [16]:
import pandas as pd
import ast
import re


# Convert genre strings to lists
nodes_english['genre_list'] = nodes_english['genres'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else [])

# Collect all unique genres
unique_genres = set(genre for sublist in nodes_english['genre_list'] for genre in sublist)

# Define the mapping from sub-genres to prominent genres
top_genres = {
    'Pop': ['pop', 'j-pop', 'k-pop', 'synthpop', 'electropop', 'teen pop'],
    'Rock': ['rock', 'punk', 'garage'],
    'Hip Hop': ['hip hop', 'rap', 'drill'],
    'Electronic': ['edm', 'house', 'techno', 'electronic', 'trance', 'dance'],
    'Jazz': ['jazz'],
    'Folk World': ['folk', 'celtic', 'world', 'americana', 'country'],
    'RandB Soul': ['soul', 'r&b', 'rhythm and blues', 'funk', 'blues'],
    'Metal': ['metal'],
    'Classical Orchestral': ['classical', 'orchestral', 'symphony'],
    'Reggae Dancehall': ['reggae', 'dub', 'riddim', 'ska'],
    'Latin': ['salsa', 'latin', 'bachata', 'cumbia'],
    'Alternative Indie': ['indie', 'alternative', 'emo'],
    'Blues': ['blues'],
    'Punk': ['punk'],
    'Soundtrack': ['soundtrack', 'anime', 'broadway', 'movie']
}

# Function to assign each unique genre to a top genre
def assign_genre(unique_genre):
    # Clean up the genre string for matching
    genre_clean = unique_genre.lower().strip()
    
    # Check against each top genre's sub-genres
    for top_genre, subgenres in top_genres.items():
        for subgenre in subgenres:
            # Use regex to check for word boundaries to avoid partial matches
            if re.search(rf'\b{subgenre}\b', genre_clean):
                return top_genre
    return 'unknown'

# Map each unique genre to a top genre
genre_mapping = {genre: assign_genre(genre) for genre in unique_genres}

# Apply this mapping to a column in your DataFrame
nodes_english['prominent_genres'] = nodes_english['genre_list'].apply(
    lambda genres: [genre_mapping.get(genre, 'Other') for genre in genres]
)

# You can now view the resultant DataFrame to check the mappings
print(nodes_english[['genres', 'prominent_genres']])

                                                   genres  \
0                          ['nordic house', 'russelater']   
1                  ['christlicher rap', 'german hip hop']   
2                                                      []   
3       ['dancehall', 'lovers rock', 'modern reggae', ...   
4       ['classic swedish pop', 'norrbotten indie', 's...   
...                                                   ...   
156417                            ['classic swedish pop']   
156418                                                 []   
156419                                                 []   
156420                                                 []   
156421                                                 []   

                                         prominent_genres  
0                                   [Electronic, unknown]  
1                                      [Hip Hop, Hip Hop]  
2                                                      []  
3       [unknown, Rock, Reg

In [17]:

unique_prominent_genres = set(gen for sublist in nodes_english['prominent_genres'] for gen in sublist)

# Count the number of unique prominent genres
unique_count = len(unique_prominent_genres)

# Print the count and the unique prominent genres
print(f"Number of unique prominent genres: {unique_count}")
print("Unique prominent genres:", unique_prominent_genres)

Number of unique prominent genres: 14
Unique prominent genres: {'Classical Orchestral', 'Metal', 'Soundtrack', 'RandB Soul', 'Folk World', 'Rock', 'Jazz', 'Electronic', 'Reggae Dancehall', 'Latin', 'Hip Hop', 'Pop', 'unknown', 'Alternative Indie'}


In [19]:
# Step 1: Explode the genre lists into separate rows
df_exploded = nodes_english.explode('prominent_genres')

# Step 2: Ensure deduplication across these exploded items per original entry
df_exploded = df_exploded.groupby(df_exploded.index).apply(lambda x: x.drop_duplicates(subset=['prominent_genres'])).reset_index(level=0, drop=True)

# Step 3: Perform one-hot encoding
one_hot_encoded = pd.get_dummies(df_exploded['prominent_genres'])

# Step 4: Sum up genre indicators for each original record by grouping back using the initial index
final_one_hot_encoded = one_hot_encoded.groupby(one_hot_encoded.index).sum()

# Combine back to original DataFrame as needed
df_final = pd.concat([nodes_english.drop(columns=['prominent_genres']), final_one_hot_encoded], axis=1)

# Display the one-hot encoded result
print(df_final)

                      name  followers  popularity  \
0                Byklubben     1738.0          24   
1                 Kontra K  1999676.0          72   
2                    Maxim    34596.0          36   
3       Christopher Martin   249233.0          52   
4            Jakob Hellman    21193.0          39   
...                    ...        ...         ...   
156417        David Urwitz     5470.0          29   
156418             Darmiko     2022.0          23   
156419            Katriell      268.0           0   
156420          Yung Fresh      181.0          19   
156421       Rakshitha Rao       23.0          24   

                                                   genres  \
0                          ['nordic house', 'russelater']   
1                  ['christlicher rap', 'german hip hop']   
2                                                      []   
3       ['dancehall', 'lovers rock', 'modern reggae', ...   
4       ['classic swedish pop', 'norrbotten indie', 's... 

In [20]:
df_final = df_final.drop(columns=['genres','genre_list'])


In [21]:
df_final.to_csv('artist_without_country_wise_rank.csv')

In [22]:
df = pd.read_csv('artist_without_country_wise_rank.csv')

In [23]:
df.shape

(143848, 18)

In [24]:
df.isnull().sum()

Unnamed: 0              0
name                    0
followers               0
popularity              0
Alternative Indie       0
Classical Orchestral    0
Electronic              0
Folk World              0
Hip Hop                 0
Jazz                    0
Latin                   0
Metal                   0
Pop                     0
RandB Soul              0
Reggae Dancehall        0
Rock                    0
Soundtrack              0
unknown                 0
dtype: int64